In [1]:
import os
import sys

os.chdir(os.path.join(os.getcwd(), '..'))

from data.datamodule import PollenDataModule
from core.models import VisualHull

from lightning.pytorch.trainer import Trainer
from torchvision import transforms

In [2]:
data_module = PollenDataModule(batch_size=12, 
                               num_workers=4,
                               image_transforms=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Resize((128, 128)),
                               ]))

data_module.setup("fit")

In [3]:
model = VisualHull()

In [4]:
trainer = Trainer(
    accelerator="auto",
    devices=1,
    max_epochs=1,
    num_sanity_val_steps=0,
    enable_progress_bar=True,
    enable_model_summary=False,
    log_every_n_steps=1,
)

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\fahrn\Documents\Classes\bat\sequoia\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [5]:
from core.metrics import init_metrics

init_metrics(model, "train")
init_metrics(model, "val")

In [6]:
trainer.fit(model, data_module)

You are using a CUDA device ('NVIDIA GeForce RTX 3070 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\fahrn\Documents\Classes\bat\sequoia\.venv\Lib\site-packages\lightning\pytorch\core\optimizer.py:183: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer


Training: |          | 0/? [00:00<?, ?it/s]

c:\Users\fahrn\Documents\Classes\bat\sequoia\.venv\Lib\site-packages\lightning\pytorch\loops\optimization\automatic.py:134: `training_step` returned `None`. If this was on purpose, ignore this warning...


Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
# ground truth 128x128 voxels need to be preprocessed under data/processed/voxels (probably torch file or something)

# during training/validation, the dataset class reads the voxels from the disk and loads them into memory

# estimators like visual hull can manually rotate their output to "revert" the rotation applied to the input images